In [1]:
import os
import pdb
import glob
import random
import string
import pandas as pd
import numpy as np
import argparse
import tqdm
import time
import h5py
import wandb
from PIL import Image
from collections import OrderedDict

import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import torch
from tensorboardX import SummaryWriter
import torchvision.models as models
from sklearn.metrics import r2_score

from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from transformers import BertTokenizerFast as BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup

import torchvision

from utils import cal_weights

C:\Users\akgo1\anaconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\akgo1\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# Included parse_args("") - quotes inside

def get_parser():
	parser = argparse.ArgumentParser()
	parser.add_argument('--name', type=str, default=''.join(random.choice(string.ascii_lowercase) for i in range(10)))
	parser.add_argument('--seed', type=int, default=7)
	parser.add_argument('--gpu', action='store_true')
	parser.add_argument('--gpus', action='store_true')

	parser.add_argument('--data_path', type=str, default='./dataset/celeba/crowd.csv')

	# data
	parser.add_argument('--train_test_split', type=float, default=0.2)
	parser.add_argument('--train_val_split', type=float, default=0.2)
	parser.add_argument('--classnum', type=int, default=0)
	parser.add_argument('--batch_size', type=int, default=32)
	parser.add_argument('--num_workers', type=int, default=0)
	parser.add_argument('--max_len', type=int, default=300)
	parser.add_argument('--max_vocab', type=int, default=1000)
	parser.add_argument('--img_size', type=int, default=256)
	parser.add_argument('--crop_size', type=int, default=224)
	parser.add_argument('--target', type=str, default='skin')

	# train
	parser.add_argument('--log', action='store_true')
	parser.add_argument('--test', action='store_true')
	parser.add_argument('--save', type=str, default='./save/nsi/base.pth')
	parser.add_argument('--epoches', type=int, default=15)
	parser.add_argument('--base_lr', type=float, default=1e-3)
	parser.add_argument('--train_frac', type=float, default=1.0)
	parser.add_argument('--weight_decay', type=float, default=2e-5)
	parser.add_argument('--dropout', type=float, default=0.1)
	parser.add_argument('--warm_up_split', type=int, default=5)

	# model
	parser.add_argument('--pretrain', action='store_true')
	parser.add_argument('--backbone', type=str, default='vgg16')
	parser.add_argument('--embed_dim', type=int, default=64)
	parser.add_argument('--hidden_dim', type=int, default=32)
	parser.add_argument('--word_embed_size', type=int, default=128)

	opt = parser.parse_args("")

	return opt

In [3]:
# Transforms images - crops random portions, converts to gray scale with a probability, flips, normalizes etc.

class CusDataset(Dataset):
	def __init__(self, opt, csv, mode='train'):
		self.opt = opt
		self.csv = csv
		self.mode = mode

		self.trans = self.get_trans()

	def __getitem__(self, idx):
		item = self.csv.iloc[idx]
		path = item['path']
		if not os.path.exists(path):
			path = path.replace('pool', 'images')
		img = Image.open(path).convert('RGB')
		img = self.trans(img)
		return img, item['label_idx'], path

	def __len__(self):
		return len(self.csv)

	def get_trans(self):
		normalize = transforms.Normalize(
			mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

		if self.mode == 'train':
			return transforms.Compose([
				transforms.RandomResizedCrop((128, 128)),
				transforms.RandomGrayscale(p=0.2),
				transforms.RandomHorizontalFlip(),
				transforms.ToTensor(),
				normalize,
			])
		else:
			return transforms.Compose([
				transforms.Resize((128, 128)),
				# transforms.CenterCrop(224),
				transforms.ToTensor(),
				normalize,
			])

In [4]:
# backbone is a std model used initially (default = 'vgg16')
# then data is passed to a linear layer and dropout layer with certain probability (--dropout)

class AttriCLS(nn.Module):
	def __init__(self, args):
		super(AttriCLS, self).__init__()
		self.opt = args
		self.backbone, out_feat = self.get_backbone()
		self.classifier = nn.Linear(out_feat, opt.classnum)
		self.dropout = nn.Dropout(opt.dropout)

	def forward(self, imgs):
		### IMAGE #####
		imgs = self.backbone(imgs)
		imgs = self.dropout(imgs.mean(-1).mean(-1))

		outs = self.classifier(imgs)

		return outs

	def get_backbone(self):
		if self.opt.backbone == 'vgg16':
			backbone = nn.Sequential(*list(torchvision.models.vgg16(pretrained=self.opt.pretrain).children())[:-1])
# 			print('inside class',backbone)
			out_feat = 512
		elif self.opt.backbone == 'resnet18':
			backbone = nn.Sequential(*list(torchvision.models.resnet18(pretrained=self.opt.pretrain).children())[:-1])
			out_feat = 512
		elif self.opt.backbone == 'resnet50':
			backbone = nn.Sequential(*list(torchvision.models.resnet50(pretrained=self.opt.pretrain).children())[:-1])
			out_feat = 2048
		elif self.opt.backbone == 'alexnet':
			backbone = nn.Sequential(*list(torchvision.models.alexnet(pretrained=self.opt.pretrain).children())[:-1])
			out_feat = 256
		elif self.opt.backbone == 'squeezenet':
			backbone = nn.Sequential(*list(torchvision.models.squeezenet1_0(pretrained=self.opt.pretrain).children())[:-1])
			out_feat = 512
		else:
			raise

		return backbone, out_feat

In [5]:
# Target refers to face components to be protected, default is 'skin'

def convert_crowd_to_label(opt, csv):
	csv = csv[csv['compo'] == opt.target].values
	res = []
# 	print("Inside func")
# 	display(csv)
	for line in csv:
		res.append([line[line[3]], line[6]]) #Appending the attribute of matching images as given by crowdsource worker
# 		print(res)
		res.append([line[line[4]], line[6]])
# 		print(res)
		res.append([line[3-line[3]-line[4]], line[7]])
# 		print(res)
# 	print(res)
	res = pd.DataFrame(res, columns=['path', 'label'])
	name2idx = {x:i for i, x in enumerate(sorted(res['label'].unique()))}
# 	print(name2idx) #Dictionary mapping label to description
	res['label_idx'] = res['label'].map(name2idx)
# 	print(res) 
	return res

In [6]:
if __name__ == '__main__':
	opt = get_parser()
# 	print(opt)
	csv = pd.read_csv(opt.data_path)
# 	display(csv)
	csv = convert_crowd_to_label(opt, csv)
	display(csv)
	csv.to_csv("List of images.csv")   
	list_of_images = csv['path'].str.split('/').str[-1]
	list_of_images.to_csv("List of images.csv")
# 	print(list_of_images)    
# 	display(images)

	opt.classnum = csv['label'].nunique()
	if opt.log:
		wandb.init(project="CompoFair", name=opt.name)
		wandb.config.update(opt)

	train, test = train_test_split(csv, test_size=opt.train_test_split, stratify=csv['label'], random_state=42)

	train_set = CusDataset(opt, train, mode='train')
	test_set = CusDataset(opt, test, mode='test')

	trainloader = DataLoader(train_set, batch_size=opt.batch_size,
							 shuffle=True, num_workers=opt.num_workers, drop_last=True)
	testloader = DataLoader(test_set, batch_size=opt.batch_size,
						   shuffle=False, num_workers=opt.num_workers)
	print(trainloader)    
    
	model = AttriCLS(opt)
	if opt.gpu:
		model = model.cuda()
        
	#Assigning weights to classes to avoid skewness of distribution
    
	loss_func = nn.CrossEntropyLoss(cal_weights(train, 'label').cuda() if opt.gpu else cal_weights(train, 'label'))
    
	params, base_params = [], []
	for name, param in model.named_parameters():
		if 'backbone' in name:
			base_params.append(param)
		else:
			params.append(param)
	print('model parameters:', name, params)
    
	optimizer = optim.AdamW([{'params': params}, {'params': base_params, 'lr': opt.base_lr/10}], lr=opt.base_lr, weight_decay=opt.weight_decay)
	total_training_steps = len(trainloader) * opt.epoches
	warmup_steps = total_training_steps // opt.warm_up_split
	scheduler = get_linear_schedule_with_warmup(
		optimizer,
		num_warmup_steps=warmup_steps,
		num_training_steps=total_training_steps
	)

	if opt.test:
		best_model, best_loss = torch.load(opt.save), -1
	else:
		best_model, best_loss = model.state_dict(), 1e5
        
		for epo in range(opt.epoches):
			model.train()
			train_loss = 0.
			for step, pack in enumerate(trainloader):
				imgs, targets, _ = pack
# 				print('imgs', imgs.size(), 'targets', targets.size()) batch_size=32             
				if opt.gpu:
					imgs, targets = imgs.cuda(), targets.cuda()
				out = model(imgs)
				loss = loss_func(out, targets)

				optimizer.zero_grad()
				loss.backward()
				optimizer.step()
				scheduler.step()

				train_loss += loss.item()

			if opt.log:
				wandb.log({'train_loss': train_loss/len(trainloader)})

			model.eval()
			eval_loss = 0.
			for step, pack in enumerate(testloader):
				imgs, targets, _ = pack
				if opt.gpu:
					imgs, targets = imgs.cuda(), targets.cuda()
				out = model(imgs)
				eval_loss += loss_func(out, targets).item()

			if opt.log:
				wandb.log({'val_loss': eval_loss/len(testloader)})

			if eval_loss < best_loss:
				best_model, best_loss = model.state_dict(), eval_loss

	model.load_state_dict(best_model)
	target_preds, target_gts, paths = [], [], []
	model.eval()
	with torch.no_grad():
		for step, pack in enumerate(testloader):
			imgs, targets, path = pack
			if opt.gpu:
				imgs, targets = imgs.cuda(), targets
			out = model(imgs)
			target_preds.append(out.cpu().numpy())
			target_gts.append(targets.numpy())
			paths.append(path)
	target_preds = np.concatenate(target_preds, 0)
	target_gts = np.concatenate(target_gts, 0)
	paths = np.concatenate(paths, 0)

	print('top1: {} top2: {}'.format(top_k_accuracy_score(target_gts, target_preds, k=1), top_k_accuracy_score(target_gts, target_preds, k=2)))

	if not opt.test:
		torch.save(model.state_dict(), './save/base_skin.pth')

,path,label,label_idx
0,./dataset/celeba/pool/053969.jpg,light,3
1,./dataset/celeba/pool/132912.jpg,light,3
2,./dataset/celeba/pool/052093.jpg,brown,0
3,./dataset/celeba/pool/016652.jpg,brown,0
4,./dataset/celeba/pool/055875.jpg,brown,0
...,...,...,...
280,./dataset/celeba/pool/094813.jpg,light,3
281,./dataset/celeba/pool/076592.jpg,brown,0
282,./dataset/celeba/pool/154034.jpg,light,3
283,./dataset/celeba/pool/076325.jpg,light,3


model parameters: classifier.bias [Parameter containing:
tensor([[-0.0075, -0.0335, -0.0369,  ...,  0.0412,  0.0079,  0.0160],
        [ 0.0063,  0.0180, -0.0107,  ...,  0.0256,  0.0397,  0.0055],
        [-0.0385,  0.0276, -0.0199,  ...,  0.0317, -0.0418, -0.0042],
        [-0.0292, -0.0046, -0.0073,  ..., -0.0224,  0.0224, -0.0242]],
       requires_grad=True), Parameter containing:
tensor([ 0.0066, -0.0001,  0.0134,  0.0437], requires_grad=True)]


KeyboardInterrupt: 

In [22]:
opt.test

False